In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import polars as pl
from itertools import combinations

In [3]:
data = pl.scan_parquet("../data/processed/wioa_data_tier2.parquet")

In [6]:
df = data.collect().to_pandas()

In [19]:
df_with_occupations = df["occupation_code_x"].dropna()

num_transportation =  len(df_with_occupations[df_with_occupations.astype(str).str.startswith("53")])
total = len(df_with_occupations)

num_transportation / total

0.11326879693275575

In [26]:
df.groupby(by=["occupation_title_x"])["occupation_code_x"].count().sort_values(ascending=False).head(20)

occupation_title_x
Customer Service Representatives                                 6913
Home Health Aides                                                4906
Stockers and Order Fillers                                       4732
Nursing Assistants                                               4476
Personal Care Aides                                              3940
Cashiers                                                         3725
General and Operations Managers                                  3637
Retail Salespersons                                              3332
Laborers and Freight, Stock, and Material Movers, Hand           3057
Janitors and Cleaners, Except Maids and Housekeeping Cleaners    2941
Light Truck Drivers                                              2866
Security Guards                                                  2795
Sales Managers                                                   2603
Heavy and Tractor-Trailer Truck Drivers                          2170
P

In [21]:
# Compute counts and percentages
result = (
    data
    .select(
        pl.col("industry_title_y")
        .value_counts()
        .alias("vc")
    )
    .unnest("vc")
    .with_columns(
        (pl.col("count") / total_count * 100).alias("percentage")
    )
    .sort("count", descending=True)
    .collect()
)

pl.Config.set_tbl_rows(len(result))

print(result)

shape: (89, 3)
┌─────────────────────────────────┬────────┬────────────┐
│ industry_title_y                ┆ count  ┆ percentage │
│ ---                             ┆ ---    ┆ ---        │
│ str                             ┆ u32    ┆ f64        │
╞═════════════════════════════════╪════════╪════════════╡
│ Administrative and support ser… ┆ 361492 ┆ 16.490197  │
│ Federal government, excluding … ┆ 185096 ┆ 8.443533   │
│ Food services and drinking pla… ┆ 141715 ┆ 6.46462    │
│ Professional, scientific, and … ┆ 113057 ┆ 5.157326   │
│ Ambulatory healthcare services  ┆ 89054  ┆ 4.06238    │
│ Social assistance               ┆ 81069  ┆ 3.698128   │
│ Specialty trade contractors     ┆ 74398  ┆ 3.393817   │
│ Educational services; state, l… ┆ 69774  ┆ 3.182884   │
│ Nursing and residential care f… ┆ 58497  ┆ 2.66846    │
│ General merchandise retailers   ┆ 51362  ┆ 2.342983   │
│ Hospitals; state, local, and p… ┆ 45865  ┆ 2.092226   │
│ Food manufacturing              ┆ 38553  ┆ 1.758674   │